In [1]:
import pandas as pd
import gensim
import numpy as np

In [2]:
recipes_df = pd.read_csv("clean_data.csv")

In [3]:
def clean_list(ingredient_list):
    ingredient_list = ingredient_list[2:-2]
    ingredient_list = ''.join(ingredient_list)
    ingredient_list = ingredient_list.split("', '")
    return ingredient_list

In [4]:
recipes_df["Clean_bag"]=recipes_df["Bag_Of_Ingredients"].map(clean_list)

In [21]:
bag_of_ingredients = recipes_df["Clean_bag"]

In [6]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))

model_wiki = gensim.downloader.load('glove-wiki-gigaword-50')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
vec_keys = model_wiki.__dict__["index_to_key"]

In [8]:
model_wiki.most_similar('chicken', topn=10)

[('meat', 0.9089884757995605),
 ('fried', 0.8994253873825073),
 ('pork', 0.892033040523529),
 ('soup', 0.869308590888977),
 ('sausage', 0.8679246306419373),
 ('roasted', 0.8635038137435913),
 ('roast', 0.8601908087730408),
 ('cooked', 0.8594754934310913),
 ('shrimp', 0.8334652185440063),
 ('beef', 0.8325141668319702)]

In [9]:
def getRecipeEmbedding(sentence):
    countFound = 0
    embeddingList = []
    for wordx in sentence:
        if wordx in vec_keys:
            vector1 = model_wiki[wordx]
            embeddingList.append(vector1)
            countFound+=1
    if countFound>0:
        return np.true_divide(sum(embeddingList), countFound)
    else:
        return np.zeros(50,)

In [10]:
def recipes_list(bag_of_ingredients):
    recipes_embed_list = []
    for i in bag_of_ingredients:
            recipes_embed_list.append(getRecipeEmbedding(i))
    return recipes_embed_list

In [22]:
recipes_embed_list = recipes_list(bag_of_ingredients)

In [12]:
from numpy import dot
from numpy.linalg import norm

def similar_recipe(ingredients_embedded):
    cos_sim = []
    for i in range(0,len(recipes_embed_list)):
        if sum(recipes_embed_list[i]) == 0.0:
            cos_sim.append(0)
        else:    
            cos_sim.append(np.dot(ingredients_embedded,recipes_embed_list[i])/(norm(ingredients_embedded)*norm(recipes_embed_list[i])))
    dis_array = np.array(cos_sim)
    return dis_array


In [27]:
ingredients = ["cheese","onion","tortilla","lettuce","sausage","ham","tomato","tuna","pesto","salsa","mustard","cucumber","potato","sweet"]

In [28]:
ingredients_embedded = getRecipeEmbedding(ingredients)

In [29]:
def getListofRecipes(cos_sim,n):
    n_index = (-cos_sim).argsort()[:n]
    titles = []
    for i in n_index:
        titles.append(recipes_df.iloc[i,1:3])
    return titles


In [30]:
pd.DataFrame(getListofRecipes(similar_recipe(ingredients_embedded),10))

,Title,Ingredients
4071,Po-Tacos,"['4 sweet or russet potatoes', 'pesto, greek y..."
12908,Stacked Enchiladas with Turkey and Chipotle Beans,"['2 10-ounce cans medium-hot enchilada sauce',..."
2296,"""Nextover"" Chicken Tacos with Quick Refried Beans",['4 cooked chicken thighs from Crispy Chicken ...
605,Smash Burger Alfresco,"['1 cup finely chopped dill pickles', '1/2 cup..."
2216,Polish-Style Pizza with Mushrooms and Chives (...,"['1 (8–9"") soft hoagie roll or small Italian l..."
9154,Gorditas,"['1 8.25-ounce can creamed corn', '1/2 teaspoo..."
8309,Poblano and Mushroom Tacos,"['2 tablespoons vegetable oil, divided', '1 fr..."
3891,Gochujang Grilled Cheese,['2 ounces grated Gruyère (about 6 tablespoons...
7003,Chicken and Cheese Sliders,"['Vegetable oil cooking spray', '1 green bell ..."
9867,Bbq Onion and Smoked Gouda Quesadillas with Pe...,"['6 tablespoons olive oil, divided', '1 pound ..."
